In [7]:
# !pip install requests
# !pip install python-dotenv

In [59]:
import requests
from dotenv import load_dotenv 
import os
import base64
import json

In [3]:
# load_dotenv loads all the local environment variables file from within the current directory (.env files)
load_dotenv()

True

In [4]:
# storing the variables from the .env file into the current session (client_id and client_secret)
client_id = <'Your CLient ID'>
client_secret = '<Your CLient Secret>'

In [15]:
def get_token():
    
    ''' Authorization Token
    In order to retrive the authorization token: 
    Headers must include- 
    An authorization string in the format: 'Authorization: Basic <base64 encoded client_id:client_secret>'
    Content-Type:  Always set to application/x-www-form-urlencoded.

    Body - 
    grant_type: Set it to 'client_credentials'.

    For more details, 
    check the Spotify API documentation: https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow 
    '''
    
    # converting the client credentials to base64 encoded string
    auth_string = client_id + ':' + client_secret
    auth_string_bytes = auth_string.encode('ascii')
    auth_base64_bytes = base64.b64encode(auth_string_bytes)
    auth_base64_string = auth_base64_bytes.decode('ascii')
    
    base_url = 'https://accounts.spotify.com/api/token'
    
    headers = {
        'Authorization': 'Basic ' + auth_base64_string,
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    body = { 'grant_type' : 'client_credentials'}
    
    # API call to create an authorization token
    result = requests.post(base_url, headers = headers, data = body) # returns a HTTP status code.
    # The client can read the result of the request in the body and the headers of the response.
    
    # loading the content of the result of the request as a json string
    json_result = json.loads(result.content)
    
    # storing the value of the access token from the json string into a token variable
    token = json_result['access_token']
    
    return token

In [58]:
def get_auth_header(token):
    ''' To use the access token you must include the following header in your API calls:
    Authorization - Valid access token following the format: Bearer <Access Token> 
    
    For more details, check the spotify documentation on making API calls: 
    https://developer.spotify.com/documentation/web-api/concepts/access-token
    '''
    token = get_token()
    return {'Authorization': 'Bearer ' + token}

In [70]:
def search_artist(token, artist_name):
    ''' Searching for artist ID, returns content for the artist as a json string
    
    GET request must contain-
    1. search query (string)
    2. type (an array of strings). Allowed values: "album", "artist", "playlist", "track", "show", "episode", "audiobook"
    
    For more details, check the spotify documentation: https://developer.spotify.com/documentation/web-api/reference/search
    '''
    
    url = 'https://api.spotify.com/v1/search' # endpoint url
    headers = get_auth_header(token)
    
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url+query
    
    result = requests.get(query_url, headers = headers)
    json_result = json.loads(result.content)['artists']['items']
    
#     json_result = json.loads(result.content)['artists']['items']
    
    if len(json_result) == 0:
        print('No artist with this name exists...')
        return None
    
    return json_result[0]

search_artist(token, "the local")


{'external_urls': {'spotify': 'https://open.spotify.com/artist/3KXxNSegk9keTDArt5YQMm'},
 'followers': {'href': None, 'total': 14663},
 'genres': ['kentucky roots', 'roots americana'],
 'href': 'https://api.spotify.com/v1/artists/3KXxNSegk9keTDArt5YQMm',
 'id': '3KXxNSegk9keTDArt5YQMm',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebaae5ac8703fe5566c23a12d9',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174aae5ac8703fe5566c23a12d9',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178aae5ac8703fe5566c23a12d9',
   'width': 160}],
 'name': 'The Local Honeys',
 'popularity': 30,
 'type': 'artist',
 'uri': 'spotify:artist:3KXxNSegk9keTDArt5YQMm'}

In [66]:
def get_artist_tracks(token, artist_id):
    '''Get the artists' top 10 tracks as a json string
    
    GET request must contain-
    1. Artist ID (string) - The Spotify ID of the artist. 
                            Example: 0TnOYISbd1XYRBk9myaseg
                            
    2. market/country(string) - country code as per 'ISO 3166-1 alpha-2 country code'
    
    For more details, check the Spotify API documentation:
    https://developer.spotify.com/documentation/web-api/reference/get-an-artists-top-tracks
    '''
    # retrive the artist id
    artist_result = search_artist(token, input("Enter an artist's name: "))
    artist_id = artist_result['id']

    artist_id_url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"    
    headers = get_auth_header(token)
    
    result = requests.get(artist_id_url, headers = headers)
    json_result = json.loads(result.content)["tracks"]
    
    return json_result
    
tracks = get_artist_tracks(token, artist_id)

for idn, track in enumerate(tracks):
    print(f"{idn+1}. {track['name']}")


Enter an artist's name: Lorde
1. Royals
2. Ribs
3. Team
4. Supercut
5. Green Light
6. Liability
7. Buzzcut Season
8. Perfect Places
9. Everybody Wants To Rule The World - From “The Hunger Games: Catching Fire” Soundtrack
10. Tennis Court
